# WTHOR棋譜データ ダウンロード

In [1]:
wthor_url = [
    'https://www.ffothello.org/wthor/base_zip/WTH_2020.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2019.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2018.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2017.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2016.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2015.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2014.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2013.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2012.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2011.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2010.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2009.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2008.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2007.ZIP',
    'https://www.ffothello.org/wthor/base_zip/WTH_2006.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_2005.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_2004.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_2003.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_2002.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_2001.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_2000.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1999.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1998.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1997.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1996.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1995.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1994.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1993.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1992.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1991.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_1990.ZIP',
    # 'https://www.ffothello.org/wthor/base_zip/WTH_7789.ZIP',
]

In [ ]:
for url in wthor_url:
    !wget $url

In [ ]:
for url in wthor_url:
    zip_name = url[url.rfind('/') + 1:]

    !unzip $zip_name -d WTHOR
    !rm $zip_name

# 棋譜データ解析

In [4]:
# バイナリ解析関数

import struct

def read_wtb_header(wtb):
    # ファイルを作成した年の上2桁
    read_data = wtb.read(1)
    year_first_two_digits = struct.unpack('b', read_data)
    # ファイルを作成した年の下2桁
    read_data = wtb.read(1)
    year_last_two_digits = struct.unpack('b', read_data)
    # ファイルを作成した月
    read_data = wtb.read(1)
    month = struct.unpack('b', read_data)
    # ファイルを作成した日
    read_data = wtb.read(1)
    day = struct.unpack('b', read_data)
    # 試合数(.wtbのみ)
    read_data = wtb.read(4)
    number_games = struct.unpack('I', read_data)
    # レコード数(.jou, .trnのみ)
    read_data = wtb.read(2)
    number_record = struct.unpack('H', read_data)
    # 試合の行われた年(.wtbのみ)
    read_data = wtb.read(2)
    match_year = struct.unpack('H', read_data)
    # 盤のサイズ(.wtbのみ)
    read_data = wtb.read(1)
    board_size = struct.unpack('b', read_data)
    # 試合タイプ(.wtbのみ)
    read_data = wtb.read(1)
    match_type = struct.unpack('b', read_data)
    # 深さ(.wtbのみ)
    read_data = wtb.read(1)
    depth = struct.unpack('b', read_data)
    # 予約領域
    read_data = wtb.read(1)
    reserve = struct.unpack('b', read_data)

    file_date = str(year_first_two_digits[0]) + str(year_last_two_digits[0]) + "/" + str(month[0]) + "/" + str(day[0])

    return file_date, number_games[0], match_year[0], board_size[0], match_type[0], depth[0]

def read_wtb_record(wtb):
    # 大会番号
    read_data = wtb.read(2)
    tournament_number = struct.unpack('h', read_data)
    # 黒番プレーヤー番号
    read_data = wtb.read(2)
    black_player_number = struct.unpack('h', read_data)
    # 白番プレーヤー番号
    read_data = wtb.read(2)
    white_player_number = struct.unpack('h', read_data)
    # 黒番の石数
    read_data = wtb.read(1)
    black_count = struct.unpack('b', read_data)
    # 黒番の石数理論値
    read_data = wtb.read(1)
    black_theoretical_value = struct.unpack('b', read_data)
    # 棋譜
    read_data = wtb.read(60)
    game_record = []
    for move in read_data:
        if (move != 0):
            move = str(move)
            game_record.append([int(move[1]) - 1, int(move[0]) - 1])
        else:
            game_record.append([0, 0])
    return tournament_number[0], black_player_number[0], white_player_number[0], black_count[0], black_theoretical_value[0], game_record

In [5]:
# 棋譜データファイル一覧

import glob

wthor_files = sorted(glob.glob('WTHOR/*.wtb'))
print(wthor_files)

['WTHOR/WTH_2006.wtb', 'WTHOR/WTH_2007.wtb', 'WTHOR/WTH_2008.wtb', 'WTHOR/WTH_2009.wtb', 'WTHOR/WTH_2010.wtb', 'WTHOR/WTH_2011.wtb', 'WTHOR/WTH_2012.wtb', 'WTHOR/WTH_2013.wtb', 'WTHOR/WTH_2014.wtb', 'WTHOR/WTH_2015.wtb', 'WTHOR/WTH_2016.wtb', 'WTHOR/WTH_2017.wtb', 'WTHOR/WTH_2018.wtb', 'WTHOR/WTH_2019.wtb', 'WTHOR/WTH_2020.wtb']


In [6]:
# 棋譜データ解析

game_records = []
total_number_games = 0
for index, wthor_file in enumerate(wthor_files):
    wtb_binary = open(wthor_file, 'rb')

    date, number_games, _, _, _, _ = read_wtb_header(wtb_binary)
    total_number_games += number_games
    print(str(index) + " " + date + ":" + str(number_games))

    for i in range(number_games):
        _, _, _, black_count, _, game_record = read_wtb_record(wtb_binary)
        game_records.append([black_count, game_record])

    wtb_binary.close()
print("Total:" + str(total_number_games))

0 2015/9/30:2942
1 2015/9/30:2478
2 2015/9/30:2232
3 2015/9/30:4348
4 2015/9/30:2172
5 2015/9/30:1891
6 2015/9/30:2208
7 2015/9/30:2396
8 2015/9/30:1817
9 2016/10/29:1926
10 2017/7/6:2013
11 2018/8/8:2449
12 2019/7/27:2429
13 2020/4/30:1949
14 2020/4/27:268
Total:33518


In [7]:
# レコード確認 [ゲーム終了時の黒石数, [棋譜]]
print(game_records[0])

[26, [[5, 4], [3, 5], [2, 2], [3, 2], [2, 3], [5, 3], [4, 5], [1, 2], [4, 1], [2, 4], [2, 5], [4, 2], [3, 1], [5, 6], [5, 1], [5, 2], [6, 2], [2, 0], [6, 3], [5, 0], [1, 4], [6, 4], [5, 5], [7, 2], [1, 3], [0, 3], [5, 7], [6, 0], [0, 2], [0, 1], [1, 1], [6, 5], [7, 5], [1, 5], [0, 5], [7, 4], [7, 3], [7, 6], [6, 6], [2, 6], [4, 6], [7, 7], [6, 7], [3, 6], [3, 0], [4, 0], [1, 6], [4, 7], [6, 1], [2, 1], [1, 7], [7, 0], [0, 0], [2, 7], [0, 4], [7, 1], [1, 0], [0, 7], [3, 7], [0, 6]]]


# 棋譜→盤面変換用 関数

In [8]:
BLANK = 0
BLACK = 1
WHITE = 2
TEMP = 3

In [9]:
# デバッグ用描画
def print_board(board):
    for row in board:
        for square in row:
            if square == BLANK:
                print('□', end='')
            if square == BLACK:
                print('●', end='')
            if square == WHITE:
                print('○', end='')
        print()

In [10]:
# ボード初期化
def initialize_board():
    board_width = 8
    board_height = 8
    board = [[0 for i in range(board_width)] for j in range(board_height)]

    board[3][3] = WHITE
    board[3][4] = BLACK
    board[4][3] = BLACK
    board[4][4] = WHITE

    return board

In [11]:
import copy

# 駒裏返し処理
def reverse_piece(current_player, board, x, y):
    direction_list = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1]]

    for direction in direction_list:
        reverse_piece_(current_player, board, x, y, direction[0], direction[1])

    return board

def reverse_piece_(current_player, board, x, y, direction_x, direction_y):
    width = len(board[0])
    height = len(board)

    reverse_count = 0
    check_x = x + direction_x
    check_y = y + direction_y

    while (0 <= check_x < width) and (0 <= check_y < height):
        if board[check_x][check_y] == current_player:
            check_x -= direction_x
            check_y -= direction_y

            while not ((check_x == x) and (check_y == y)):
                board[check_x][check_y] = current_player
                check_x -= direction_x
                check_y -= direction_y
                reverse_count += 1

            if reverse_count > 0:
                board[x][y] = current_player;
            break
        elif board[check_x][check_y] == BLANK:
            break
        check_x += direction_x
        check_y += direction_y

    return board

# パス判定
def check_pass(current_player, board):
    width = len(board[0])
    height = len(board)

    for y in range(height):
        for x in range(width):
            if board[x][y] == BLANK:
                temp_board = copy.deepcopy(board)
                temp_board = reverse_piece(current_player, temp_board, x, y)
                if temp_board[x][y] == current_player:
                    return False
    return True

# 駒カウント
def count_piece(board):
    width = len(board[0])
    height = len(board)

    black_count = 0
    white_count = 0

    for y in range(height):
        for x in range(width):
            if board[x][y] == BLACK:
                black_count += 1
            if board[x][y] == WHITE:
                white_count += 1
    return black_count, white_count

In [12]:
import copy

# 棋譜再生
def play(board, record, debug_print=True):
    width = len(board[0])
    height = len(board)

    x_data = []
    y_data = []

    player = BLACK

    for move in record:
        prev_board = copy.deepcopy(board)
        board = reverse_piece(player, board, move[0], move[1])

        # 駒が置くことが出来た場合、学習用データに追加しプレイヤーを入れ替える
        if board[move[0]][move[1]] == player:
            # 駒を置くプレイヤーを1、敵プレイヤーを2に統一する
            if player == WHITE:
                for index, row in enumerate(prev_board):
                    l_replace = [TEMP if i == BLACK else i for i in row]
                    l_replace = [BLACK if i == WHITE else i for i in l_replace]
                    l_replace = [WHITE if i == TEMP else i for i in l_replace]
                    prev_board[index] = l_replace
            # 学習データ追加
            x_data.append(prev_board)

            # 学習データ追加
            # x_data.append([player, prev_board])
            y_data_board = [[0 for i in range(width)] for j in range(height)]
            y_data_board[move[0]][move[1]] = 1
            y_data.append(y_data_board)

            # プレイヤー入れ替え
            player = (WHITE if player == BLACK else BLACK)

            if debug_print:
                print(move)
                print_board(board)

            if check_pass(player, board):
                player = (WHITE if player == BLACK else BLACK)

                # 両プレイヤーがパスを行った場合、ゲーム終了
                if check_pass(player, board):
                    break
    return board, x_data, y_data

# 棋譜→盤面変換

In [13]:
# 1つ目：盤面初期化
board = initialize_board()
print_board(board)

□□□□□□□□
□□□□□□□□
□□□□□□□□
□□□○●□□□
□□□●○□□□
□□□□□□□□
□□□□□□□□
□□□□□□□□


In [14]:
# 1つ目：棋譜再生、および学習データ作成
X_dataset = []
y_dataset = []

record = game_records[0][1]
board, x_data, y_data = play(board, record)
X_dataset.append(x_data)
y_dataset.append(y_data)

black_count, _ = count_piece(board)
is_correct = True if game_records[0][0] == black_count else False
print(is_correct, str(game_records[0][0]), black_count)

[5, 4]
□□□□□□□□
□□□□□□□□
□□□□□□□□
□□□○●□□□
□□□●●□□□
□□□□●□□□
□□□□□□□□
□□□□□□□□
[3, 5]
□□□□□□□□
□□□□□□□□
□□□□□□□□
□□□○○○□□
□□□●●□□□
□□□□●□□□
□□□□□□□□
□□□□□□□□
[2, 2]
□□□□□□□□
□□□□□□□□
□□●□□□□□
□□□●○○□□
□□□●●□□□
□□□□●□□□
□□□□□□□□
□□□□□□□□
[3, 2]
□□□□□□□□
□□□□□□□□
□□●□□□□□
□□○○○○□□
□□□●●□□□
□□□□●□□□
□□□□□□□□
□□□□□□□□
[2, 3]
□□□□□□□□
□□□□□□□□
□□●●□□□□
□□○●○○□□
□□□●●□□□
□□□□●□□□
□□□□□□□□
□□□□□□□□
[5, 3]
□□□□□□□□
□□□□□□□□
□□●●□□□□
□□○●○○□□
□□□●○□□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[4, 5]
□□□□□□□□
□□□□□□□□
□□●●□□□□
□□○●●○□□
□□□●●●□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[1, 2]
□□□□□□□□
□□○□□□□□
□□○●□□□□
□□○●●○□□
□□□●●●□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[4, 1]
□□□□□□□□
□□○□□□□□
□□○●□□□□
□□●●●○□□
□●□●●●□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[2, 4]
□□□□□□□□
□□○□□□□□
□□○○○□□□
□□●●●○□□
□●□●●●□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[2, 5]
□□□□□□□□
□□○□□□□□
□□○○○●□□
□□●●●●□□
□●□●●●□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[4, 2]
□□□□□□□□
□□○□□□□□
□□○○○●□□
□□○○●●□□
□●○●●●□□
□□□○●□□□
□□□□□□□□
□□□□□□□□
[3, 1]
□□□□□□□□
□□○□□□□□
□□○○○●□□
□●●●●●□□
□●○●●●□□


In [15]:
# 2つ目以降の棋譜の変換

from tqdm import tqdm

for record in tqdm(game_records[1:]):
    board = initialize_board()
    board, x_data, y_data = play(board, record[1], False)
    X_dataset.append(x_data)
    y_dataset.append(y_data)

100%|██████████| 33517/33517 [18:44<00:00, 29.80it/s]


In [16]:
import numpy as np

# npyで保存
np.save('X_dataset_', X_dataset)
np.save('y_dataset_', y_dataset)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [17]:
import gc
del game_records
gc.collect()

103

# 学習用データに成形

In [18]:
import collections

def flatten(l):
    for el in l:
        if isinstance(el, collections.abc.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

In [19]:
# 多次元配列を1次元化
x_temp_list = []

for x_data in tqdm(X_dataset):
    for x_temp in x_data:
        x_temp_list.append(list(flatten(x_temp)))

X_dataset = np.array(x_temp_list)

del x_temp_list
gc.collect()

100%|██████████| 33518/33518 [01:40<00:00, 331.94it/s]


0

In [20]:
# 多次元配列を1次元化  
y_temp_list = []
for y_data in tqdm(y_dataset):
    for y_temp in y_data:
        y_temp_list.append(list(flatten(y_temp)))
        
y_dataset = np.array(y_temp_list)

del y_temp_list
gc.collect()

100%|██████████| 33518/33518 [01:46<00:00, 314.54it/s]


0

In [21]:
print(len(X_dataset))

2005741


In [22]:
# npyで保存
np.save('X_dataset', X_dataset)
np.save('y_dataset', y_dataset)

# Googleドライブに保存

In [23]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


In [24]:
!mkdir '/content/gdrive/MyDrive/ReversiTrainData'

In [25]:
!cp 'X_dataset.npy' '/content/gdrive/MyDrive/ReversiTrainData'
!cp 'y_dataset.npy' '/content/gdrive/MyDrive/ReversiTrainData'